In [1]:
#| default_exp main

In [2]:
#| hide
%load_ext autoreload
%autoreload 2

In [3]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [4]:
#| export
import os, torch, scipy.sparse as sp, joblib, argparse, pickle, numpy as np, inspect
from typing import Optional, Dict, Callable, Union, List

from tqdm.auto import tqdm
from transformers import DistilBertConfig

from xcai.sdata import SMainXCDataset, SXCDataBlock, SXCDataset
from xcai.data import XCDataBlock, XCDataset, XCCollator
from xcai.block import NXCBlock, SXCBlock, XCBlock, CFGS
from xcai.core import Info, get_best_model, get_last_model, load_config
from xcai.transform import AugmentMetaInputIdsTfm
from xcai.learner import XCLearningArguments, XCLearner

from xcai.models.PPP0XX import DBT024
from xcai.models.distillation import TCH001
from xcai.models.classifiers import CLS001
from xcai.clustering.cluster import BalancedClusters, get_cluster_size

from sugar.core import *

from xclib.utils.sparse import retain_topk

## `Arguement`

In [5]:
#| export
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--build_block', action='store_true')
    parser.add_argument('--use_pretrained', action='store_true')
    
    parser.add_argument('--do_train_inference', action='store_true')
    parser.add_argument('--do_test_inference', action='store_true')
    
    parser.add_argument('--save_train_prediction', action='store_true')
    parser.add_argument('--save_test_prediction', action='store_true')
    parser.add_argument('--save_label_prediction', action='store_true')
    
    parser.add_argument('--save_representation', action='store_true')
    
    parser.add_argument('--use_sxc_sampler', action='store_true')
    parser.add_argument('--only_test', action='store_true')
    parser.add_argument('--text_mode', action='store_true')
    parser.add_argument('--use_oracle', action='store_true')

    parser.add_argument('--score_data_lbl', action='store_true')
    parser.add_argument('--score_data_meta', action='store_true')
    parser.add_argument('--score_lbl_meta', action='store_true')

    parser.add_argument('--pickle_dir', type=str, default=None)
    
    parser.add_argument('--prediction_suffix', type=str, default='')
    parser.add_argument('--save_dir_name', type=str, default=None)

    parser.add_argument('--exact', action='store_true')
    parser.add_argument('--dataset', type=str)

    parser.add_argument('--num_meta_cluster', type=int, default=1)
    parser.add_argument('--normalize', action='store_true')
    parser.add_argument('--use_ln', action='store_true')
    
    return parser.parse_known_args()[0]
    

In [6]:
#| export
def check_inference_mode(args):
    return args.do_train_inference or args.do_test_inference or args.save_train_prediction or args.save_test_prediction or args.save_representation or args.score_data_lbl or args.score_data_meta or args.score_lbl_meta
    

## Clustering

In [13]:
#| export
def get_metadata_representation(mname:str, meta_info:Dict, collator:XCCollator, normalize:Optional[bool]=True, 
                                use_layer_norm:Optional[bool]=True, use_encoder_parallel:Optional[bool]=True):
    dset = SXCDataset(SMainXCDataset(data_info=meta_info))
    model = DBT024.from_pretrained(mname, margin=0.3, tau=0.1, n_negatives=10, apply_softmax=True, 
                                   normalize=normalize, use_layer_norm=use_layer_norm, 
                                   use_encoder_parallel=use_encoder_parallel)
    model.init_dr_head()
    
    args = XCLearningArguments(
        output_dir='.',
        per_device_train_batch_size=1024,
        per_device_eval_batch_size=800,
        representation_num_beams=200,
        representation_accumulation_steps=10,
        representation_search_type='BRUTEFORCE',
        
        target_indices_key='plbl2data_idx',
        target_pointer_key='plbl2data_data2ptr',
        
        use_encoder_parallel=True,
        max_grad_norm=None,
        fp16=True,
        
        use_cpu_for_searching=True,
        use_cpu_for_clustering=True,
    )
    
    learn = XCLearner(
        model=model,
        args=args,
        eval_dataset=dset,
        data_collator=collator,
    )
    return learn._get_data_representation(dset, to_cpu=True)
    

In [14]:
#| export
def _get_cluster_mapping(meta_repr:Optional[Union[torch.Tensor,str]]=None, cluster_sz:Optional[int]=3, mname:Optional[str]=None, 
                        meta_info:Optional[Dict]=None, collator:Optional[XCCollator]=None, normalize:Optional[bool]=True, 
                        use_layer_norm:Optional[bool]=True, use_encoder_parallel:Optional[bool]=True):
    if meta_repr is None: 
        meta_repr = get_metadata_representation(mname, meta_info, collator, normalize=normalize, use_layer_norm=use_layer_norm)
    elif isinstance(meta_repr, str):
        meta_repr = torch.load(meta_repr)
            
    if cluster_sz > 1:
        clusters = BalancedClusters.proc(meta_repr.half(), min_cluster_sz=cluster_sz)
    
        metadata_idx2cluster = torch.zeros(meta_repr.shape[0], dtype=torch.int64)
        for i,o in enumerate(clusters): metadata_idx2cluster[o] = i
    
        meta_repr = torch.vstack([meta_repr[o].mean(dim=0) for o in clusters])
        num_meta_clusters = len(clusters)
    else:
        metadata_idx2cluster = torch.arange(meta_repr.shape[0])
        num_meta_clusters = meta_repr.shape[0]

    return metadata_idx2cluster, meta_repr, num_meta_clusters
    

In [15]:
#| export
def get_cluster_mapping(do_inference:bool, use_pretrained:bool, num_metadata:int, cluster_size:int, meta_embed_init_file:Optional[str]=None, 
                        model_name:Optional[str]=None, meta_info:Optional[Dict]=None, collator:Optional[Callable]=None, 
                        normalize:Optional[bool]=True, use_layer_norm:Optional[bool]=None, output_dir:Optional[str]=None, **kwargs):
    if do_inference and not use_pretrained:
        metadata_idx2cluster, meta_repr = None, None
        num_meta_cluster = get_cluster_size(num_metadata, cluster_size) if cluster_size > 1 else num_metadata
    else:
        map_file = None if output_dir is None else f'{output_dir}/cluster_info.pth'
        if map_file is not None and os.path.exists(map_file):
            metadata_idx2cluster, meta_repr, num_meta_cluster = torch.load(map_file)
        else:
            metadata_idx2cluster, meta_repr, num_meta_cluster = _get_cluster_mapping(meta_embed_init_file, cluster_sz=cluster_size, 
                                                                                     mname=model_name, meta_info=meta_info, 
                                                                                     collator=collator, normalize=normalize, 
                                                                                     use_layer_norm=use_layer_norm)
            if output_dir is not None:
                os.makedirs(output_dir, exist_ok=True)
                torch.save((metadata_idx2cluster, meta_repr, num_meta_cluster), map_file)
            
    return metadata_idx2cluster, meta_repr, num_meta_cluster
    

## Build `block`

In [7]:
#| export
class Filter:

    @staticmethod
    def topk(data_lbl:sp.csr_matrix, k:Optional[int]=3):
        return retain_topk(data_lbl, k=k)

    @staticmethod
    def threshold(data_lbl:sp.csr_matrix, t:int):
        idx = np.where(data_lbl.data < t)[0]
        data_lbl.data[idx] = 0
        data_lbl.eliminate_zeros()
        return data_lbl

    @staticmethod
    def difference(data_lbl:sp.csr_matrix, t:int):
        rowise_max = data_lbl.max(axis=1).toarray().ravel()
        scores = np.repeat(rowise_max, np.diff(data_lbl.indptr)) - data_lbl.data
        data_lbl.data[scores > t] = 0
        data_lbl.eliminate_zeros()
        return data_lbl
        

In [8]:
#| export
def filter_metadata_dset(dset, topk:Union[int,Dict]=5, abs_thresh:Union[float,Dict]=0.2, 
                         diff_thresh:Union[float,Dict]=0.3):
    for meta_name in dset.meta.keys():
        data_meta, lbl_meta = dset.meta[meta_name].data_meta, dset.meta[meta_name].lbl_meta
        update_meta = False
        
        k = topk.get(meta_name, None) if isinstance(topk, dict) else topk
        if k is not None: 
            data_meta = retain_topk(data_meta, k=k)
            if lbl_meta is not None: lbl_meta = retain_topk(lbl_meta, k=k)
            update_meta = True

        t = abs_thresh.get(meta_name, None) if isinstance(abs_thresh, dict) else abs_thresh
        if t is not None:
            data_meta = Filter.threshold(data_meta, t=t)
            if lbl_meta is not None: lbl_meta = Filter.threshold(lbl_meta, t=t)
            update_meta = True

        t = diff_thresh.get(meta_name, None) if isinstance(diff_thresh, dict) else diff_thresh
        if t is not None:
            data_meta = Filter.difference(data_meta, t=t)
            if lbl_meta is not None: lbl_meta = Filter.difference(lbl_meta, t=t)
            update_meta = True
            
        if update_meta:
            dset.meta[meta_name].update_meta_matrix(data_meta, lbl_meta)
            if dset.meta[meta_name].use_meta_distribution or dset.meta[meta_name].return_scores:
                dset.meta[meta_name]._store_scores() 
    

In [9]:
#| export
def filter_metadata_block(block, train_topk:Union[int,Dict]=5, train_abs_thresh:Union[float,Dict]=0.2, 
                          train_diff_thresh:Union[float,Dict]=0.3, test_topk:Union[int,Dict]=5, 
                          test_abs_thresh:Union[float,Dict]=0.2, test_diff_thresh:Union[float,Dict]=0.3):
    if block.train is not None: 
        filter_metadata_dset(block.train.dset, train_topk, train_abs_thresh, train_diff_thresh)
    if block.test is not None:
        filter_metadata_dset(block.test.dset, test_topk, test_abs_thresh, test_diff_thresh)
                    

In [10]:
#| export
def retain_topk_metadata(block, train_k:Union[int,Dict]=5, test_k:Union[int,Dict]=3):
    if train_k is not None and block.train is not None:
        for meta_name in block.train.dset.meta.keys():
            k = train_k.get(meta_name, None) if isinstance(train_k, dict) else train_k
            if k is not None:
                data_meta = retain_topk(block.train.dset.meta[meta_name].data_meta, k=k)
                lbl_meta = block.train.dset.meta[meta_name].lbl_meta
                block.train.dset.meta[meta_name].update_meta_matrix(data_meta, lbl_meta)
                if block.train.dset.meta[meta_name].use_meta_distribution or block.train.dset.meta[meta_name].return_scores: 
                    block.train.dset.meta[meta_name]._store_scores()

    if test_k is not None and block.test is not None:
        for meta_name in block.test.dset.meta.keys():
            k = test_k.get(meta_name, None) if isinstance(test_k, dict) else test_k
            if k is not None:
                data_meta = retain_topk(block.test.dset.meta[meta_name].data_meta, k=k)
                lbl_meta = block.test.dset.meta[meta_name].lbl_meta
                block.test.dset.meta[meta_name].update_meta_matrix(data_meta, lbl_meta)
                if block.test.dset.meta[meta_name].use_meta_distribution or block.test.dset.meta[meta_name].return_scores: 
                    block.test.dset.meta[meta_name]._store_scores()

def retain_topk_labels(block, train_k:int=5, test_k:int=3):
    if train_k is not None and block.train is not None:
        block.train.dset.data.data_lbl = retain_topk(block.train.dset.data.data_lbl, k=train_k)
        block.train.dset.data._store_indices()
        if block.train.dset.data.use_main_distribution or block.train.dset.data.return_scores: 
            block.train.dset.data._store_scores()
            
    if test_k is not None and block.test is not None:
        block.test.dset.data.data_lbl = retain_topk(block.test.dset.data.data_lbl, k=test_k)
        block.test.dset.data._store_indices()
        if block.test.dset.data.use_main_distribution or block.test.dset.data.return_scores:
            block.test.dset.data._store_scores()
    

In [11]:
#| export
def get_valid_dset(block):
    num_empty_idx = sum(block.dset.data.data_lbl.getnnz(axis=1) == 0)
    return block._getitems(np.where(block.dset.data.data_lbl.getnnz(axis=1) > 0)[0]) if num_empty_idx > 0 else block
    

In [12]:
#| export
def get_config(config:Union[str, Dict], config_key:Optional[str]=None, data_dir:Optional[str]=None, **kwargs):
    if isinstance(config, str) and os.path.exists(config):
        config = load_config(config, config_key)
    elif isinstance(config, str):
        config = CFGS[config](data_dir)[config_key]
    elif isinstance(config, dict):
        pass
    else: raise ValueError(f'Invalid configuration: {config}')
        
    for k in config['parameters']:
        if k in kwargs:config['parameters'][k]=kwargs.pop(k)
    return config
    

In [13]:
#| export
def tokenize_info(info:Dict, config:Dict, max_sequence_length:int):
    tokz, tokz_args = Info(), {p:config[p] for p in inspect.signature(Info.tokenize).parameters if p in config}
    tokz.info = info
    tokz.tokenize(**tokz_args, max_sequence_length=max_sequence_length)
    

In [14]:
#| export
def augment_metadata(dset:Union[XCDataset,SXCDataset], meta_name:str, config:Union[str, Dict], 
                     config_key:Optional[str]=None, data_dir:Optional[str]=None, 
                     prompt:Optional[Callable]=None, sep_tok:Optional[str]=" :: ", **kwargs):
    if prompt is None: prompt = lambda x:x
        
    text = [prompt(o) for o in dset.data_info['input_text']]
    data_meta = dset.meta[meta_name].data_meta
    meta_info = dset.meta[meta_name].meta_info

    aug_text = []
    for p,q,txt in tqdm(zip(data_meta.indptr, data_meta.indptr[1:], text), total=len(text)):
        aug_text.append(txt + sep_tok.join([meta_info['input_text'][i] for i in data_meta.indices[p:q]]))

    config = get_config(config, config_key=config_key, data_dir=data_dir)
    dset.data.data_info['input_text'] = aug_text
    tokenize_info(dset.data.data_info, config['parameters'], 
                  max_sequence_length=config['parameters']['main_max_data_sequence_length'])
    

In [15]:
#| export
def get_pkl_file(pkl_dir:str, fname:str, use_sxc_sampler:bool, use_exact:Optional[bool]=False, 
                 use_only_test:Optional[bool]=False, use_oracle:Optional[bool]=False, 
                 use_text_mode:Optional[bool]=False):
    pkl_file = f'{pkl_dir}/{fname}'

    if use_sxc_sampler: pkl_file = f'{pkl_file}_sxc'
    else: pkl_file = f'{pkl_file}_xcs'
            
    if use_exact: pkl_file = f'{pkl_file}_exact'
    if use_only_test: pkl_file = f'{pkl_file}_only-test'
    if use_oracle: pkl_file = f'{pkl_file}_oracle'
    if use_text_mode: pkl_file = f'{pkl_file}_text-mode'

    return f'{pkl_file}.joblib'
    

In [16]:
#| export
def build_block(pkl_file:str, config:Union[str,Dict], use_sxc:Optional[bool]=True, config_key:Optional[str]=None, 
                do_build:Optional[bool]=False, only_test:Optional[bool]=False, use_oracle:Optional[bool]=False, 
                remove_empty_datapoints:Optional[bool]=False, train_label_topk:Optional[int]=None, test_label_topk:Optional[int]=None, 
                
                train_meta_topk:Optional[int]=None, test_meta_topk:Optional[int]=None, train_meta_abs_thresh:Optional[float]=None, 
                test_meta_abs_thresh:Optional[float]=None, train_meta_diff_thresh:Optional[float]=None, test_meta_diff_thresh:Optional[float]=None,
                
                
                aug_meta_name:Optional[str]=None, aug_data_seq_length:Optional[int]=128, aug_lbl_seq_length:Optional[int]=128, 
                aug_exclude_sep:Optional[bool]=False, perform_data_meta_aug:Optional[bool]=False, perform_lbl_meta_aug:Optional[bool]=False, 
                prompt:Optional[Callable]=None, data_dir:Optional[str]=None, **kwargs):

    if not os.path.exists(pkl_file): do_build = True

    if do_build:
        if isinstance(config, str) and os.path.exists(config): 
            config = load_config(config, config_key)
            if only_test and 'train' in config['path']:
                if 'lbl_info' not in config['path']['test']:
                    config['path']['test']['lbl_info'] = config['path']['train']['lbl_info']
                del config['path']['train'] 
    
        if use_sxc:
            block = SXCBlock.from_cfg(config, config_key, padding=True, return_tensors='pt', data_dir=data_dir, **kwargs)
            if use_oracle:
                if block.train is not None and aug_meta_name in block.train.dset.meta:
                    augment_metadata(dset=block.train.dset, meta_name=aug_meta_name, config=config, 
                                     config_key=config_key, data_dir=data_dir, prompt=prompt, padding=True, return_tensors='pt')
                if block.test is not None and aug_meta_name in block.test.dset.meta:
                    augment_metadata(dset=block.test.dset, meta_name=aug_meta_name, config=config, 
                                     config_key=config_key, data_dir=data_dir, prompt=prompt, padding=True, return_tensors='pt')
        else: 
            block = XCBlock.from_cfg(config, config_key, transform_type='xcs', data_dir=data_dir, **kwargs)

            if perform_data_meta_aug: block = AugmentMetaInputIdsTfm.apply(block, aug_meta_name, 'data', aug_data_seq_length, aug_exclude_sep)
            if perform_lbl_meta_aug: block = AugmentMetaInputIdsTfm.apply(block, aug_meta_name, 'lbl', aug_lbl_seq_length, aug_exclude_sep)
            
        joblib.dump(block, pkl_file)
    else:
        block = joblib.load(pkl_file)

        def set_data_attribute(obj):
            for k in [o for o in vars(obj).keys() if not o.startswith('__')]: 
                if k in kwargs: setattr(obj, k, kwargs[k])

        def get_meta_args(kwargs, meta_name, key):
            return kwargs[key][meta_name] if isinstance(kwargs[key], dict) and meta_name in kwargs[key] else kwargs[key]

        def set_meta_attribute(obj, meta_name):
            for k in [o for o in vars(obj).keys() if not o.startswith('__')]:
                if k in kwargs: setattr(obj, k, get_meta_args(kwargs, meta_name, k))

        def set_block_attribute(bkl):            
            if bkl is not None: 
                set_data_attribute(bkl.dset.data)
                if bkl.dset.data.data_lbl_scores is None and (bkl.dset.data.use_main_distribution or bkl.dset.data.return_scores): 
                    bkl.dset.data._store_scores()
                    
                for k in bkl.dset.meta: 
                    set_meta_attribute(bkl.dset.meta[k], k)
                    if bkl.dset.meta[k].data_meta_scores is None and (bkl.dset.meta[k].use_meta_distribution or bkl.dset.meta[k].return_scores): 
                        bkl.dset.meta[k]._store_scores()

        set_block_attribute(block.train)
        set_block_attribute(block.test)
                        
    if remove_empty_datapoints: 
        block = type(block)(train=get_valid_dset(block.train), test=get_valid_dset(block.test))

    if train_label_topk is not None or test_label_topk is not None:
        retain_topk_labels(block, train_k=train_label_topk, test_k=test_label_topk)

    if (
        train_meta_topk is not None or test_meta_topk is not None or
        train_meta_abs_thresh is not None or test_meta_abs_thresh is not None or
        train_meta_diff_thresh is not None or test_meta_diff_thresh is not None
    ):
        filter_metadata_block(block, train_topk=train_meta_topk, train_abs_thresh=train_meta_abs_thresh, 
                              train_diff_thresh=train_meta_diff_thresh, test_topk=test_meta_topk, 
                              test_abs_thresh=test_meta_abs_thresh, test_diff_thresh=test_meta_diff_thresh)
        # retain_topk_metadata(block, train_k=train_meta_topk, test_k=test_meta_topk)

    return block
        

## Load `model`

In [8]:
#| export
def raw_mapping(src_file, targ_file):
    src_ids, src_txt = load_raw_file(src_file)
    targ_ids, targ_txt = load_raw_file(targ_file)

    assert set(src_ids).difference(targ_ids) == 0
    
    src2targ_idx = torch.zeros(len(targ_ids), dtype=torch.long)
    
    targ_ids2idx = {k:i for i,k in enumerate(targ_ids)}
    for i,id in enumerate(src_ids): src2targ_idx[i] = targ_ids2idx[id]


In [12]:
#| export
def load_model(output_dir:str, model_fn:Callable, model_args:Dict, init_fn:Optional[Callable]=None, 
               init_args:Optional[Dict]=dict(), do_inference:Optional[bool]=False, use_pretrained:Optional[bool]=False, 
               type:Optional[str]="best"):
    if do_inference:
        os.environ["WANDB_MODE"] = "disabled"
        if not use_pretrained:
            fname = get_best_model(output_dir) if type == "best" else get_last_model(output_dir)
            model_args["mname"] = f'{output_dir}/{os.path.basename(fname)}'

    model = model_fn(**model_args)
    
    if (not do_inference or use_pretrained) and (init_fn is not None): 
        init_fn(model, **init_args)

    return model
    

## `main` function

In [71]:
#| export
def get_output(pred_idx:torch.Tensor, pred_ptr:torch.Tensor, pred_score:torch.Tensor, n_lbl:int, **kwargs):
    n_data = pred_ptr.shape[0]
    pred_ptr = torch.cat([torch.zeros((1,), dtype=torch.long), pred_ptr.cumsum(dim=0)])
    pred = sp.csr_matrix((pred_score,pred_idx,pred_ptr), shape=(n_data, n_lbl))
    return pred
    

In [9]:
#| export
def main(learn, args, n_lbl:int, eval_dataset=None, train_dataset=None, eval_k:int=None, train_k:int=None, 
         save_teacher:bool=False, save_classifier:bool=False, resume_from_checkpoint:Optional[bool]=None, 
         save_dir:Optional[str]=None, save_dir_name:Optional[str]=None, metadata_name:Optional[str]=None):
    eval_dataset = learn.eval_dataset if eval_dataset is None else eval_dataset
    train_dataset = learn.train_dataset if train_dataset is None else train_dataset
    
    do_infer = check_inference_mode(args)
    
    if do_infer:
        trn_repr = tst_repr = lbl_repr = trn_pred = tst_pred = None
        prediction_suffix = f'_{args.prediction_suffix}' if len(args.prediction_suffix) else ''

        save_dir_name = 'predictions' if save_dir_name is None else save_dir_name
        pred_dir = f'{learn.args.output_dir}/{save_dir_name}' if save_dir is None else f'{save_dir}/{save_dir_name}'
        os.makedirs(pred_dir, exist_ok=True)

        if args.save_representation:
            trn_repr = None if train_dataset is None else learn._get_data_representation(train_dataset, to_cpu=True)
            tst_repr = None if eval_dataset is None else learn._get_data_representation(eval_dataset, to_cpu=True)
            lbl_repr = learn._get_lbl_representation(eval_dataset if train_dataset is None else train_dataset, to_cpu=True)

            if trn_repr is not None: torch.save(trn_repr, f'{pred_dir}/train_repr{prediction_suffix}.pth')
            if tst_repr is not None: torch.save(tst_repr, f'{pred_dir}/test_repr{prediction_suffix}.pth')
            if lbl_repr is not None: torch.save(lbl_repr, f'{pred_dir}/label_repr{prediction_suffix}.pth')

            if save_teacher:
                config = TCHConfig(n_data=trn_repr.shape[0], n_lbl=lbl_repr.shape[0])
                teacher = TCH001(config)
                teacher.init_embeddings(trn_repr, lbl_repr)
                teacher.freeze_embeddings()
                teacher.save_pretrained(f'{learn.args.output_dir}/teacher')

            if save_classifier:
                classifier = CLS001(DistilBertConfig(), n_train=trn_repr.shape[0], n_test=tst_repr.shape[0], n_lbl=lbl_repr.shape[0])
                classifier.init_representation(trn_repr, tst_repr, lbl_repr)
                classifier.freeze_representation()
                classifier.save_pretrained(f'{learn.args.output_dir}/representation')

        if args.score_data_lbl or args.score_data_meta or args.score_lbl_meta:
            trn_repr, lbl_repr = learn.get_data_and_lbl_representation(train_dataset)
            tst_repr = learn._get_data_representation(eval_dataset)

            save_dir = f'{learn.args.output_dir}/matrices' if save_dir is None else f'{save_dir}/matrices'
            os.makedirs(save_dir, exist_ok=True)
            
            if args.score_data_lbl:
                trn_lbl = train_dataset.data.score_data_lbl(trn_repr, lbl_repr, batch_size=1024, normalize=args.normalize)
                if trn_lbl is not None: sp.save_npz(f"{save_dir}/trn_lbl{prediction_suffix}.npz", trn_lbl)
                
                tst_lbl = eval_dataset.data.score_data_lbl(tst_repr, lbl_repr, batch_size=1024, normalize=args.normalize)
                if tst_lbl is not None: sp.save_npz(f"{save_dir}/tst_lbl{prediction_suffix}.npz", tst_lbl)

            if args.score_data_meta or args.score_lbl_meta:
                meta_name = f'{learn.args.data_aug_meta_name}_meta' if metadata_name is None else f'{metadata_name}_meta'
                meta_repr = learn._get_metadata_representation(eval_dataset if meta_name in eval_dataset.meta else train_dataset, meta_name=metadata_name)
                if args.score_data_meta:
                    if meta_name in train_dataset.meta:
                        trn_meta = train_dataset.meta[meta_name].score_data_meta(trn_repr, meta_repr, batch_size=1024, normalize=args.normalize)
                        if trn_meta is not None: sp.save_npz(f"{save_dir}/trn_meta{prediction_suffix}.npz", trn_meta)

                    if meta_name in eval_dataset.meta:
                        tst_meta = eval_dataset.meta[meta_name].score_data_meta(tst_repr, meta_repr, batch_size=1024, normalize=args.normalize)
                        if tst_meta is not None: sp.save_npz(f"{save_dir}/tst_meta{prediction_suffix}.npz", tst_meta)

                if args.score_lbl_meta:
                    dset = train_dataset if meta_name in train_dataset.meta else eval_dataset
                    lbl_meta = dset.meta[meta_name].score_lbl_meta(lbl_repr, meta_repr, batch_size=1024, normalize=args.normalize)
                    if lbl_meta is not None: sp.save_npz(f"{save_dir}/lbl_meta{prediction_suffix}.npz", lbl_meta)
                    
        if args.do_test_inference:
            o = learn.predict(eval_dataset)
            print(o.metrics)

            tst_pred = get_output(o.pred_idx, o.pred_ptr, o.pred_score, n_lbl=n_lbl)
            if eval_k is not None: tst_pred = retain_topk(tst_pred, k=eval_k)

            if args.save_test_prediction:
                with open(f'{pred_dir}/test_predictions{prediction_suffix}.pkl', 'wb') as file:
                    pickle.dump(o, file)
                sp.save_npz(f'{pred_dir}/test_predictions{prediction_suffix}.npz', tst_pred)

        if args.do_train_inference:
            o = learn.predict(train_dataset)
            print(o.metrics)

            trn_pred = get_output(o.pred_idx, o.pred_ptr, o.pred_score, n_lbl=n_lbl)
            if train_k is not None: trn_pred = retain_topk(trn_pred, k=train_k)
            
            if args.save_train_prediction:
                with open(f'{pred_dir}/train_predictions{prediction_suffix}.pkl', 'wb') as file:
                    pickle.dump(o, file)
                sp.save_npz(f'{pred_dir}/train_predictions{prediction_suffix}.npz', trn_pred)
                
        return trn_repr, tst_repr, lbl_repr, trn_pred, tst_pred
    else:
        learn.train(resume_from_checkpoint)
        